In [13]:
import numpy as np
import json
from munch import Munch
import itertools
from collections import defaultdict

In [14]:
# process_list = []
# names = ['apt','bob','sally']
# for name in names:
#     with open(f'{name}.json','r') as file:
#         process_list.append(json.load(file))

In [181]:
names = ['apt','bob','sally']
apt_hmm, bob_hmm, sally_hmm = process_load(names)

In [183]:
def process_load(names_list, folder_path = '', delay = True):
    '''
    Build the hmm object.
    Also, reading in the json converts tuples to string. need to convert back to tuples.
    '''
    process_list = []
    for names in names_list:
        file_path = folder_path + f'{names}.json'
        with open(file_path,'r') as file:
            process = json.load(file)

        if delay:
            mu = process['mu']
            tprob = {eval(k): round((1-mu)*v,5) for k,v in process['transition_probs'].items()}
        else:
            tprob = {eval(k): v for k,v in process['transition_probs'].items()}

        #Augment the user emissions so they're also of the form (Server, (Server,Action))
        if names.startswith('apt'):
            eprob = {eval(k): v for k,v in process['emission_probs'].items()}
        else:
            eprob = {}
            for k, v in process['emission_probs'].items():
                key = eval(k)
                if key[1] is None:
                    eprob[key[0],None] = v
                else:
                    eprob[(key[0],key)] = v
        
        states = set()
        emits = set()
        
        for k in tprob.keys():
            states.update(k)
        for k in eprob.keys():
            emits.add(k[1])

        states = list(states)
        
        if delay:
            for k in list(states): #need to create a separate copy since we're appending to states
                wait_state = f'WAIT_{k}'
                tprob[k,wait_state] = mu
                for j in states:
                    if (k,j) in tprob.keys():
                        tprob[wait_state,j] = tprob[k,j] # 1 - mu factor already applied
                        tprob[wait_state,wait_state] = mu
                eprob[wait_state,None] = 1.
                states.append(wait_state)
            emits.add(None)

        emits = list(emits)
        
        process_hmm = Munch(name = names, states = states, emits = emits, tprob = tprob, \
                           eprob = eprob, initprob = process['start_probs'])

        if delay:
            process_hmm.mu = process['mu']

        process_list.append(process_hmm)

    return process_list


In [223]:
def hmm2numpy(hmm, ix_list = None, return_ix = False):
    '''
    Converts/generates relevant parameters/weights into numpy arrays for Baum-Welch.
    By assumption, the update/emission parameters associated with the constraint are static.
    For now, fix the emission probabilities.
    Only the hmm paramters are being optimized.
    '''
    #Initialize and convert all quantities  to np.arrays

    if ix_list:
        state_ix, emit_ix = ix_list
    else:
        state_ix = {s: i for i, s in enumerate(hmm.states)}
        emit_ix = {s: i for i, s in enumerate(hmm.emits)}

    K = len(state_ix)
    M = len(emit_ix)
    #Compute the hmm parameters
    tmat = np.zeros((K,K))
    init_prob = np.zeros(K)
    emat = np.zeros((K,M))

    #Initial distribution. 
    for i in hmm.states:
        if i not in hmm.initprob:
            continue
        init_prob[state_ix[i]] = hmm.initprob[i]

    #Transition matrix
    for i in hmm.states:
        for j in hmm.states:
            if (i,j) not in hmm.tprob:
                continue
            tmat[state_ix[i],state_ix[j]] = hmm.tprob[i,j]

    
    #Emission matrix
    for i in hmm.states:
        for m in hmm.emits:
            if (i,m) not in hmm.eprob:
                continue
            emat[state_ix[i],emit_ix[m]] = hmm.eprob[i,m]

    hmm_params = [init_prob, tmat, emat]

    if return_ix:
        return hmm_params, [state_ix, emit_ix] 
    return hmm_params


In [224]:
def hmm2numpy_apt(hmm, ix_list = None, return_ix = False):
    '''
    Converts/generates relevant parameters/weights into numpy arrays for Baum-Welch.
    By assumption, the update/emission parameters associated with the constraint are static.
    For now, fix the emission probabilities.
    Only the hmm paramters are being optimized.
    '''
    #Initialize and convert all quantities  to np.arrays
    state_ix = {s: i for i, s in enumerate(hmm.states)}

    if ix_list:
        emit_ix = ix_list[1]
    else:
        emit_ix = {s: i for i, s in enumerate(hmm.emits)}


    K, M = len(state_ix), len(emit_ix)
    #Compute the hmm parameters
    tmat = np.zeros((K,K))
    init_prob = np.zeros(K)
    emat = np.zeros((K,M))

    #Initial distribution. 
    for i in hmm.states:
        if i not in hmm.initprob:
            continue
        init_prob[state_ix[i]] = hmm.initprob[i]

    #Transition matrix
    for i in hmm.states:
        for j in hmm.states:
            if (i,j) not in hmm.tprob:
                continue
            tmat[state_ix[i],state_ix[j]] = hmm.tprob[i,j]

    
    #Emission matrix
    for i in hmm.states:
        for m in hmm.emits:
            if (i,m) not in hmm.eprob:
                continue
            emat[state_ix[i],emit_ix[m]] = hmm.eprob[i,m]

    hmm_params = [init_prob, tmat, emat]

    if return_ix:
        return hmm_params, [state_ix, emit_ix] 
    return hmm_params


In [237]:
def numpy2hmm(hmm_params, ix_list, tol = 1e-7, time_inhom = False):
    '''
    If time_inhom is true, then emat is assumed to be a list of matrices.
    '''
    state_ix, emit_ix = ix_list
    init_prob, tmat, emat = hmm_params
    initprob = {}
    tprob = {}
    eprob = {}
    K, M = len(state_ix), len(emit_ix)

    #reverse the dicts, so indices map to states
    state_ix = {v:k for k,v in state_ix.items()}
    emit_ix = {v:k for k,v in emit_ix.items()}
    #initprob
    for i in range(K):
        val = init_prob[i].item()
        if abs(val) > tol:
            initprob[state_ix[i]] = val

    for i in range(K):
        for j in range(K):
            val = tmat[i,j].item()
            if abs(val) > tol:
                tprob[state_ix[i],state_ix[j]] = val

    #eprob
    if time_inhom:
        for t in range(len(emat)):
            for i in range(K):
                for j in range(M):
                    val = emat[t][i,j].item()
                    if abs(val) > tol:
                        eprob[t,state_ix[i],emit_ix[j]] = val
    else:
        for i in range(K):
            for j in range(M):
                val = emat[i,j].item()
                if abs(val) > tol:
                    eprob[state_ix[i],emit_ix[j]] = val
                
    return initprob, tprob, eprob
                

In [187]:
def forward_marginals(hmm_params, length):
    '''
    Given an hmm, computes the sequence of marginal hidden-emission probabilities.
    IN
    hmm_params:
    1. k x k matrix of hidden transiions
    2. k x n emission matrix. 
    3. k start probabilities
    All are numpy arrays. Row is start, col is end.

    ix_list: list of state and emits dictionaries that map all hidden/emits to indices.
    
    
    '''
    init_prob, tmat, emat = hmm_params
    hidden_marginal = init_prob
    emit_marginal = [hidden_marginal @ emat]
    for t in range(1,length):
        hidden_marginal = hidden_marginal @ tmat
        emit_marginal.append(hidden_marginal @ emat)
        
    return emit_marginal

In [188]:
def simulation(hmm_params,time, homogenous = True):
    '''
    generates a full run for specified time.
    homogenous is True if the emission probs are time-homogenous.
    transition matrix aways assumed to be homogenous
    '''
    def random_draw(p):
        '''
        p is a 1D np array. 
        single random draw from probability vector p and encode as 1-hot.
        '''
        n = len(p)
        draw = np.random.choice(n,p=p)
        one_hot = np.zeros(n, dtype = int)
        one_hot[draw] = 1
        return one_hot


    init_prob, tmat, emat = hmm_params
    #Generate (X1,Y1)
    x_prev = random_draw(init_prob)
    x_list = [x_prev]
    if homogenous:
        y_list = [random_draw(x_prev @ emat)]
    else:
        y_list = [random_draw(x_prev @ emat[0])]

    #Generate rest
    for t in range(1,time):
        x_curr = random_draw(x_prev @ tmat)
        if homogenous:
            y_curr = random_draw(x_curr @ emat)
        else:
            y_curr = random_draw(x_curr @ emat[t])
        x_list.append(x_curr)
        y_list.append(y_curr)
        x_prev = x_curr

    return x_list, y_list


In [189]:
def create_combined_ix(apt_hmm, user_list):
    '''
    Generate a combined index dictionary for all possible servers and actions.
    '''
    combined_servers = set()
    combined_emits = set(apt_hmm.emits)
    
    for user in user_list:
        combined_servers.update(set(user.states))
        combined_emits.update(set(user.emits))
        
    #Generate the combined indices
    combined_server_ix = {s:i for i,s in enumerate(list(combined_servers))}
    combined_emits_ix = {s:i for i,s in enumerate(list(combined_emits))}

    return [combined_server_ix,combined_emits_ix]
    

In [198]:
def lapt_mixture(apt_hmm, user_list, length, mix_weights = None, return_ix = False):
    '''
    IN 
    apt_hmm: Munch object of the apt
    user_list: list of user hmms.
    length: int. how long we run all 3 processes.
    mix_weights: optional argument to supply np.array of mixture weights. If empty, then by default will just do uniform mixture.
        The weight for the original apt emat will be 1 - sum(mix_weights)
    add_delay: Boolean on whether the delay should be incorporated in the model.
    '''

    #Get unified indexing for all servers and emits
    usr_state_ix, emit_ix = create_combined_ix(apt_hmm, user_list)

    #Convert dicts to numpy arrays
    apt_params, apt_ix_list = hmm2numpy_apt(apt_hmm, ix_list = [usr_state_ix, emit_ix], return_ix = True)
    apt_state_ix = apt_ix_list[0]
    user_params = []
    for user in user_list:
        user_params.append(hmm2numpy(user, ix_list = [usr_state_ix, emit_ix]))

    #Compute the marginals over time
    user_marg_list = []
    for params in user_params:
        user_marg_list.append(forward_marginals(params, length))
    

    #Generate the weights
    n = len(user_marg_list)

    if mix_weights is None:
        mix_weights = np.array(1/(n+1)).repeat(n+1)
    emat_w = 1 - mix_weights.sum()

    mix_emat = []

    init_prob, tmat, emat = apt_params
    
    for t in range(length):
        curr_emat = emat_w*emat
        for i in range(n):
            curr_emat += mix_weights[i]*user_marg_list[i][t]
        mix_emat.append(curr_emat)

    initprob, tprob, eprob = numpy2hmm([init_prob, tmat, mix_emat], [apt_state_ix, emit_ix], tol = 1e-7, time_inhom = True)

    apt_hmm.eprob = eprob
    
    if return_ix:
        return apt_hmm, [apt_state_ix, emit_ix]
    return apt_hmm

In [200]:
names = ['apt','bob','sally']
apt_hmm, bob_hmm, sally_hmm = process_load(names)

In [238]:
length  = 5
mix_weights = None

usr_state_ix, emit_ix = create_combined_ix(apt_hmm, user_list)

#Convert dicts to numpy arrays
apt_params, apt_ix_list = hmm2numpy_apt(apt_hmm, ix_list = [usr_state_ix, emit_ix], return_ix = True)
apt_state_ix = apt_ix_list[0]
user_params = []
for user in user_list:
    user_params.append(hmm2numpy(user, ix_list = [usr_state_ix, emit_ix]))

#Compute the marginals over time
user_marg_list = []
for params in user_params:
    user_marg_list.append(forward_marginals(params, length))


#Generate the weights
n = len(user_marg_list)

if mix_weights is None:
    mix_weights = np.array(1/(n+1)).repeat(n+1)
emat_w = 1 - mix_weights.sum()

mix_emat = []

init_prob, tmat, emat = apt_params

for t in range(length):
    curr_emat = emat_w*emat
    for i in range(n):
        curr_emat += mix_weights[i]*user_marg_list[i][t] #broadcasting will a same marginal to each row
    mix_emat.append(curr_emat)


In [239]:
apt_hmm, ix_list = lapt_mixture(apt_hmm, user_list, 5, mix_weights = None, return_ix = True)